In [1]:
import math
from typing import List, Optional, Tuple, Union
from abc import ABC, abstractmethod

import datasets
import torch
import numpy as np
import torch.nn as nn
import logging
import copy
import gc

from datasets import load_dataset
from tqdm import tqdm
from transformers import (
    PreTrainedModel,
    PretrainedConfig,
    AutoConfig,
    AutoModelForCausalLM,
    LlamaForCausalLM,
    LlamaConfig,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    HfArgumentParser
)

from modeling_qwen2 import (
    Qwen2RMSNorm, 
    Qwen2RotaryEmbedding, 
    Qwen2MLP, 
    Qwen2Attention, 
    Qwen2FlashAttention2, 
    Qwen2SdpaAttention, 
    Qwen2DecoderLayer, 
    Qwen2PreTrainedModel, 
    Qwen2Model, 
    Qwen2ForCausalLM,
)

from configuration_qwen2 import Qwen2Config

from transformers.modeling_outputs import (
    BaseModelOutputWithPast,
)

# Configure logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [2]:
from transformers import GenerationConfig, TextStreamer
def generate(prompt, model, tokenizer, max_new_tokens=1024):
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    model.eval()
    with torch.no_grad():
        generation_config = GenerationConfig(
            repetition_penalty=1.13,
            max_new_tokens=max_new_tokens,
            temperature=0.4,
            top_p=0.95,
            # top_k=20,
            # bos_token_id=tokenizer.bos_token_id,
            # eos_token_id=tokenizer.eos_token_id,
            # eos_token_id=0, # for open-end generation.
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,
            use_cache=True,
            return_dict_in_generate=True,
            output_attentions=False,
            output_hidden_states=False,
            output_scores=False,
        )
        streamer = TextStreamer(tokenizer, skip_prompt=True)
        generated = model.generate(
            inputs=input_ids,
            generation_config=generation_config,
            streamer=streamer,
        )
    gen_tokens = generated["sequences"].cpu()[:, len(input_ids[0]):]
    output = tokenizer.batch_decode(gen_tokens)[0]
    output = output.split(tokenizer.eos_token)[0]
    return output.strip()

def get_logits(text, model, tokenizer):
    input_ids = tokenizer(text, return_tensors="pt").to(model.device)
    model.eval()
    with torch.no_grad():
        logits = model(**input_ids).logits
    return logits

def get_hidden_states(text, model, tokenizer):
    input_ids = tokenizer(text, return_tensors="pt").to(model.device)
    model.eval()
    with torch.no_grad():
        outputs = model(**input_ids, output_hidden_states=True, use_cache=False)
    return outputs

In [3]:
def free_memory():
    if not torch.cuda.is_available():
        logger.info("CUDA is not available. No GPU memory to free.")
        return
        
    initial_memory = torch.cuda.memory_allocated()
    logger.info(f"Initial GPU memory allocated: {initial_memory / 1024**3:.2f} GB")
    gc.collect()
    torch.cuda.empty_cache()

    final_memory = torch.cuda.memory_allocated()
    logger.info(f"Final GPU memory allocated: {final_memory / 1024**3:.2f} GB")

    freed_memory = initial_memory - final_memory
    logger.info(f"Freed GPU memory: {freed_memory / 1024**3:.2f} GB")

In [4]:
class MaskConfig(PretrainedConfig):
    def __init__(
        self,
        mode: str = None,
        value: Union[float, torch.Tensor] = None,
        size: torch.Size = None,
        **kwargs,
    ):
        self.mode = mode
        self.value = value
        self.size = size
        super().__init__(**kwargs)

class Mask(nn.Module):
    def __init__(self, mask_config: MaskConfig):
        super().__init__()
        self.config = mask_config
        self.size = mask_config.size
        assert self.size is not None, "Mask size must be specified."

        value = mask_config.value
        if value is not None:
            logger.warning("Highly reccommend initializing mask value using dedicated setup functions.")
            if not isinstance(value, torch.Tensor): 
                try: value = torch.tensor(value)
                except: raise ValueError(
                    f"Unable to convert {value} to torch.Tensor required for initializing a mask's weight."
                )

        ## TODO: might refactor later: modify _get_ones() to handle scalar mode.
        if mask_config.mode == "scalar":
            self.weight = nn.Parameter(value if value is not None else torch.tensor(1.0))
            
        elif mask_config.mode in ("vector_input", "vector_output"):
            ones = self._get_ones(mask_config.mode)
            self.weight = nn.Parameter(value if value is not None else ones)
        else:
            raise ValueError(f"Unsupported mask mode: {mask_config.mode}")

        self._check_shape_compatibility()

    def _get_ones(self, mode: str) -> torch.Tensor:
        """Generates a tensor of ones based on mode and size."""
        dim = 0 if mode == "vector_output" else -1
        features = self.size[dim]
        if len(self.size) == 2 and mode == "vector_output":
            return torch.ones(features, 1)
        else:
            return torch.ones(features)
          

    def _check_shape_compatibility(self):
        """Raises ValueError if the mask shape is incompatible with its size."""
        try:
            in_test = torch.rand(self.size)
            out_test = self.weight * in_test
            assert out_test.shape == in_test.shape, (
                "After applying mask, the shape of input weight does not stay the same."
            )
        except RuntimeError:
            raise ValueError("Mask initialized with an incompatible shape.")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.size != x.shape:
            logger.warning("Warning: Input shape does not match mask shape.")
        return x * self.weight

In [5]:
class ModuleWithMask(nn.Module, ABC):
    def __init__(self, *args, **kwargs):
        super(ModuleWithMask, self).__init__()

    @abstractmethod
    def forward(self, x):
        pass

class ModulesWithMasks(nn.Module, ABC):
    def __init__(self, *args, **kwargs):
        super(ModulesWithMasks, self).__init__()

    @abstractmethod
    def forward(self, x):
        pass
        
    @abstractmethod
    def get_raw_masks(self):
        pass
        
    @abstractmethod
    def get_constrained_masks(self):
        pass

In [6]:
# in_features = 24
# out_features = 50
# lin1 = nn.Linear(in_features, out_features, bias=False)
# lin2 = nn.Linear(in_features, out_features, bias=False)

In [7]:
# dim = 1
# a = lin1.weight.data * lin2.weight.data
# torch.sum(a, dim=0, keepdim=True)

In [8]:
# sum(lin1.weight.data[:, 0] * lin2.weight.data[:, 0])

In [9]:
# v = lin1.weight.data
# v, v[0, :], v[:, 0]
# v.shape

In [10]:
# a = torch.rand(5) * torch.rand(5)
# torch.sum(a, dim=0, keepdim=True)

In [11]:
# torch.sigmoid(torch.rand(5))
# torch.relu(torch.rand(5))

In [12]:
# torch.exp(torch.rand(5))

In [13]:
# t = torch.rand(4, 10)
# torch.linalg.norm(t, dim=1)

In [14]:
matA = torch.rand(100)
matB = torch.rand(100)
matA_copy = copy.deepcopy(matA)

In [15]:
1.0 - matB[matA > 0.5]

tensor([0.6741, 0.1711, 0.5974, 0.3019, 0.9672, 0.7683, 0.3308, 0.7451, 0.5241,
        0.8044, 0.5500, 0.9136, 0.7400, 0.5687, 0.8815, 0.0657, 0.4802, 0.9516,
        0.3936, 0.2414, 0.0261, 0.8712, 0.8873, 0.1255, 0.6767, 0.5501, 0.8307,
        0.3270, 0.4710, 0.9692, 0.9657, 0.7312, 0.8076, 0.4714, 0.1524, 0.4081,
        0.5837, 0.3426, 0.2297, 0.2457, 0.6571, 0.5360, 0.7971, 0.0036, 0.6654,
        0.2135, 0.7384, 0.8731])

In [16]:
matA[matA > 0.5] = matB[matA > 0.5]

In [17]:
matA_copy, matA, matB

(tensor([0.2206, 0.3439, 0.1407, 0.5551, 0.2672, 0.1888, 0.8869, 0.0304, 0.9999,
         0.2613, 0.5469, 0.9766, 0.9065, 0.0146, 0.9638, 0.5337, 0.3959, 0.2575,
         0.4650, 0.9838, 0.7129, 0.5259, 0.4179, 0.4801, 0.6235, 0.8457, 0.6211,
         0.0321, 0.9890, 0.1953, 0.3933, 0.3536, 0.2316, 0.1127, 0.5594, 0.8165,
         0.5239, 0.4155, 0.6179, 0.7754, 0.1138, 0.1025, 0.0402, 0.5737, 0.9803,
         0.1187, 0.1869, 0.8070, 0.3714, 0.9106, 0.8100, 0.8576, 0.7380, 0.5233,
         0.2840, 0.0325, 0.4044, 0.3358, 0.6512, 0.8136, 0.3398, 0.4277, 0.9060,
         0.3366, 0.2294, 0.7693, 0.0273, 0.8963, 0.1149, 0.2811, 0.5735, 0.5730,
         0.0121, 0.5207, 0.1622, 0.1276, 0.3076, 0.7074, 0.1647, 0.9598, 0.1301,
         0.7086, 0.5945, 0.9492, 0.7007, 0.2592, 0.5445, 0.8006, 0.7574, 0.3885,
         0.2551, 0.3058, 0.3216, 0.7212, 0.5213, 0.4841, 0.9975, 0.4249, 0.3248,
         0.0745]),
 tensor([0.2206, 0.3439, 0.1407, 0.3259, 0.2672, 0.1888, 0.8289, 0.0304, 0.4026,
         

In [18]:
class Constrainer(nn.Module):

    def __init__(self, component_weights, constrain_mode, mask_mode):
        super().__init__()
        self.statistics = None
        self.constrain_mode = constrain_mode
        self.mask_mode = mask_mode
        if self.constrain_mode not in ("identity", "01", "-11", "spherical"):
            raise ValueError(f"Does not support {self.constrain_mode} constraint yet!")
            
        if (self.constrain_mode == "spherical" and all([w is not None for w in component_weights])):
            self._get_spherical_stats(component_weights)

    def _get_spherical_stats(
        self, 
        component_weights: List[torch.Tensor], 
        DOT_THRESHOLD: float = 0.9995
    ):
        with torch.no_grad():
            if any([w is None for w in component_weights]):
                raise ValueError("Spherical constraint (SLERP) does not support None weights.")
            if len(component_weights) != 2: 
                raise ValueError(
                    "Spherical constraint (SLERP) only supports 2 component weights, " +
                    f"{len(component_weights)} components found."
                )

            dim = 0 if self.mask_mode in ("vector_input") else None
            v0 = self._normalize(component_weights[0], dim=dim)
            v1 = self._normalize(component_weights[1], dim=dim)
            self.dots = torch.sum(v0 * v1, dim=dim, keepdim=False) ## (out_features, in_features) -> (in_features, )
            self.theta_0s = torch.arccos(self.dots)
            self.sin_theta_0s = torch.sin(self.theta_0s)
            
    def _normalize(self, v, dim, eps: float = 1e-8):
        norm_v = torch.linalg.norm(v, dim=dim)
        norm_v[norm_v < eps] = 1.0
        v = v / norm_v
        return v

    def _constrain_identity(self, mask_weights: List[torch.Tensor]):
        return mask_weights

    def _constrain_0_1(self, mask_weights: List[torch.Tensor]):
        W = [torch.exp(w) for w in mask_weights]
        W = [w / sum(W) for w in W]
        return W

    def _constrain_neg1_1(self, mask_weights: List[torch.Tensor]):
        return mask_weights

    def _constrain_spherical(self, mask_weights: List[torch.Tensor], DOT_THRESHOLD: float = 0.9995):
        assert len(mask_weights) == 2, (
            "Spherical constraint (SLERP) only supports 2 mask weights"
        )
        
        # Transform raw masks to factor t's.
        ## QUESTION MASK: Does this modify mask_weights in-place? I only 
        ## update them via backprop.
        W = [torch.exp(w) for w in mask_weights]
        T = W[0] / sum(W)

        # Angle at timestep t's
        theta_ts = self.theta_0s * T
        sin_theta_ts = torch.sin(theta_ts)

        # Finish calculating slerp factors
        S0 = torch.sin(self.theta_0s - theta_ts) / self.sin_theta_0s
        S1 = sin_theta_ts / self.sin_theta_0s

        # Avoid NaN
        S0[self.dots > DOT_THRESHOLD] = 1 - T[self.dots > DOT_THRESHOLD]
        S1[self.dots > DOT_THRESHOLD] = T[self.dots > DOT_THRESHOLD]
        
        return [S0, S1]
        
    def forward(self, mask_weights: List[torch.Tensor]):
        if any([w is None for w in mask_weights]):
            return mask_weights
            
        if self.constrain_mode == "identity":
            return self._constrain_identity(mask_weights)
        elif self.constrain_mode == "01":
            return self._constrain_0_1(mask_weights)
        elif self.constrain_mode == "-11":
            return self._constrain_neg1_1(mask_weights)
        elif self.constrain_mode == "spherical":
            return self._constrain_spherical(mask_weights)
        else:
            raise ValueError(f"Does not support {self.constrain_mode} constraint yet!")

In [19]:
class LinearsWithMasks(ModulesWithMasks):
    def __init__(
        self,
        linears: List[nn.Linear],
        weight_modes: List[str] = None,
        weight_values: List[float] = None,
        bias_modes: List[str] = None,
        bias_values: List[float] = None,
        constrain_mode: str = "identity",
    ):
        super().__init__()

        if not all(isinstance(linear, nn.Linear) for linear in linears):
            raise ValueError("All elements in 'linears' must be instances of nn.Linear.")

        if weight_values is None or len(weight_values) != len(linears):
            raise ValueError(
                f"Weight values for masks: {weight_values} do not match with linear layers: {linears}"
            )
        if bias_values is None:
            bias_values = [None] * len(linears)
        if len(bias_values) != len(linears):
            raise ValueError(
                f"Bias values for masks: {bias_values} do not match with linear layers: {linears}"
            )

        self.linears = nn.ModuleList(linears)
        self.constrain_mode = constrain_mode

        self.weight_masks = nn.ModuleList([
            Mask(MaskConfig(mode, value, linear.weight.shape))
            for mode, value, linear in zip(weight_modes, weight_values, linears)
        ])
        
        self.weight_masks_constrainer = Constrainer(
            component_weights=[x.weight for x in self.linears], 
            constrain_mode=constrain_mode, mask_mode=weight_modes[0]
        )

        self.bias_masks = nn.ModuleList([
            Mask(MaskConfig(mode, value, linear.bias.shape)) if linear.bias is not None else None
            for mode, value, linear in zip(bias_modes, bias_values, linears)
        ])
        
        self.bias_masks_constrainer = Constrainer(
            component_weights=[x.bias if x.bias is not None else None for x in self.linears],
            constrain_mode=constrain_mode, mask_mode=bias_modes[0]
        )

    def forward(self, x):
        constrained_weight_masks = self.weight_masks_constrainer([m.weight for m in self.weight_masks])
        masked_weights = [
            w_mask * linear.weight for w_mask, linear in zip(constrained_weight_masks, self.linears)
        ]
        merged_weight = sum(masked_weights)

        constrained_bias_masks = self.bias_masks_constrainer(
            [m.weight if m is not None else None for m in self.bias_masks]
        )
        masked_biases = [
            b_mask * linear.bias if linear.bias is not None and b_mask is not None else linear.bias
            for b_mask, linear in zip(constrained_bias_masks, self.linears)
        ]

        merged_bias = (
            sum(b if b is not None else torch.zeros_like(merged_weight[:, 0]) for b in masked_biases)
            if not all(b is None for b in masked_biases) else None
        )

        return nn.functional.linear(x, merged_weight, merged_bias)

    def get_raw_masks(self):
        with torch.no_grad():
            return {
                "weight_masks": [m.weight for m in self.weight_masks],
                "bias_masks": [m.weight if m is not None else None for m in self.bias_masks],
            }

    def get_constrained_masks(self):
        with torch.no_grad():
            constrained_weight_masks = self.weight_masks_constrainer(
                [m.weight for m in self.weight_masks]
            )
            constrained_bias_masks = self.bias_masks_constrainer(
                [m.weight if m is not None else None for m in self.bias_masks]
            )
            return {
                "weight_masks": constrained_weight_masks,
                "bias_masks": constrained_bias_masks,
            }

In [20]:
# def slerp(
#     t: Union[float, np.ndarray],
#     v0: Union[np.ndarray, torch.Tensor],
#     v1: Union[np.ndarray, torch.Tensor],
#     DOT_THRESHOLD: float = 0.9995,
#     eps: float = 1e-8,
# ):
#     """
#     Spherical linear interpolation

#     From: https://gist.github.com/dvschultz/3af50c40df002da3b751efab1daddf2c
#     Args:
#         t (float/np.ndarray): Float value between 0.0 and 1.0
#         v0 (np.ndarray): Starting vector
#         v1 (np.ndarray): Final vector
#         DOT_THRESHOLD (float): Threshold for considering the two vectors as
#                                colinear. Not recommended to alter this.
#     Returns:
#         v2 (np.ndarray): Interpolation vector between v0 and v1
#     """
#     is_torch = False
#     if not isinstance(v0, np.ndarray):
#         is_torch = True
#         v0 = v0.detach().cpu().float().numpy()
#     if not isinstance(v1, np.ndarray):
#         is_torch = True
#         v1 = v1.detach().cpu().float().numpy()

#     # Copy the vectors to reuse them later
#     v0_copy = np.copy(v0)
#     v1_copy = np.copy(v1)

#     # Normalize the vectors to get the directions and angles
#     v0 = normalize(v0, eps)
#     v1 = normalize(v1, eps)
#     # import ipdb; ipdb.set_trace()

#     # Dot product with the normalized vectors (can't use np.dot in W)
#     dot = np.sum(v0 * v1)

#     # If absolute value of dot product is almost 1, vectors are ~colinear, so use lerp
#     if np.abs(dot) > DOT_THRESHOLD:
#         res = lerp(t, v0_copy, v1_copy)
#         return maybe_torch(res, is_torch)

#     # Calculate initial angle between v0 and v1
#     theta_0 = np.arccos(dot)
#     sin_theta_0 = np.sin(theta_0)

#     # Angle at timestep t
#     theta_t = theta_0 * t
#     sin_theta_t = np.sin(theta_t)

#     # Finish the slerp algorithm
#     s0 = np.sin(theta_0 - theta_t) / sin_theta_0
#     s1 = sin_theta_t / sin_theta_0
#     return [s0, s1]
#     # return [dot, theta_0, theta_t, s0, s1]

# def maybe_torch(v: np.ndarray, is_torch: bool):
#     if is_torch:
#         return torch.from_numpy(v)
#     return v


# def normalize(v: np.ndarray, eps: float):
#     norm_v = np.linalg.norm(v)
#     if norm_v > eps:
#         v = v / norm_v
#     return v

In [21]:
# from masks import LinearsWithMasks as LinearsWithMasksRef

# input_size = 20
# output_size = 40
# num_components = 2

# linears_with_bias = [nn.Linear(input_size, output_size, bias=True) for _ in range(num_components)]
# # Test without bias
# linears_without_bias = [nn.Linear(input_size, output_size, bias=False) for _ in range(num_components)]

# x = torch.rand(1, input_size)

# # for _ in range(10):  # Reduced number of iterations for faster testing
# weight_values = np.random.rand(num_components).tolist()
# weight_values = [None] * num_components
# bias_values = np.random.rand(num_components).tolist()
# bias_values = [None] * num_components

# # Test with bias
# masked_linears = LinearsWithMasks(
#     linears=linears_without_bias,
#     weight_modes=["vector_input"] * num_components,
#     weight_values=weight_values,
#     bias_modes=["scalar"] * num_components,
#     bias_values=bias_values,
#     constrain_mode = "spherical"
# )

# # masked_linears_ref = LinearsWithMasksRef(
# #     linears=linears_without_bias,
# #     weight_modes=["vector_input"] * num_components,
# #     weight_values=weight_values,
# #     bias_modes=["vectovector_inputr_input"] * num_components,
# #     bias_values=bias_values,
# # )

In [22]:
# torch.testing.assert_close(masked_linears(x), masked_linears_ref(x), atol=0, rtol=0)

In [23]:
# masked_linears(x)

In [24]:
# masked_linears.get_constrained_masks()["weight_masks"][1].numpy()

In [25]:
# v0 = linears_without_bias[0].weight.data[:, 1]
# v1 = linears_without_bias[1].weight.data[:, 1]
# slerp(
#     0.5, v0, v1
# )

In [26]:
# output_size == v0.shape[0]

In [27]:
# torch.sum(v0 * v1).data.numpy() 

In [28]:
# np.sum(v0.numpy() * v1.numpy())

In [29]:
# linears_without_bias[1].weight.data[1], linears_without_bias[0].weight.data[1]

In [30]:
class RMSNormsWithMasks(ModulesWithMasks):
    def __init__(
        self,
        rms_norms: List[nn.Module],
        modes: List[str] = None,
        values: List[float] = None,
        constrain_mode: str = "identity",
    ):
        super().__init__()
        sizes = [norm.weight.shape for norm in rms_norms]
        if any([mode != "scalar" for mode in modes]):
            logger.warning(
                f"Though you want to make a masks of modes {modes} " + \
                "for RMSNorms' weights, by default a mask only accepts a scalar mask. " + \
                "Converting modes to `scalar`."
            )
            modes = ["scalar"] * len(modes)
            
        if values is None or len(values) != len(rms_norms):
            raise ValueError(f"values for masks: {values} do not match with RMSNorm layers: {rms_norms}")

        self.rms_norms = nn.ModuleList(rms_norms)
        self.masks = nn.ModuleList([
            Mask(MaskConfig(mode, value, norm.weight.shape))
            for mode, value, norm in zip(modes, values, rms_norms)
        ])
        self.masks_constrainer = Constrainer(
            component_weights=[norm.weight for norm in self.rms_norms], 
            constrain_mode=constrain_mode, mask_mode=modes[0]
        )

    def forward(self, hidden_states):
        constrained_masks = self.masks_constrainer([m.weight for m in self.masks])
        masked_weights = [mask * norm.weight for mask, norm in zip(constrained_masks, self.rms_norms)]
        merged_weight = sum(masked_weights)
        variance_epsilon = self.rms_norms[0].variance_epsilon
        for norm in self.rms_norms:
            assert variance_epsilon == norm.variance_epsilon, ("Variance epsilon among models must be consistent")
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + variance_epsilon)
        return merged_weight * hidden_states.to(input_dtype)

    def get_raw_masks(self):
        with torch.no_grad():
            return {"masks": [m.weight for m in self.masks]}

    def get_constrained_masks(self):
        with torch.no_grad():
            constrained_masks = self.masks_constrainer([m.weight for m in self.masks])
            return {"masks": constrained_masks}

In [31]:
# hidden_size = 100
# num_components = 2
# rms_norms = [Qwen2RMSNorm(hidden_size) for _ in range(num_components)]
# for rms_norm in rms_norms:
#     rms_norm.weight.data = torch.rand(hidden_size)
# x = torch.rand(2, 4, hidden_size)

In [32]:
# from masks import RMSNormsWithMasks as RMSNormsWithMasksRef
# values = np.random.rand(num_components).tolist()
# # values = [None] * num_components

# there = RMSNormsWithMasksRef(
#     rms_norms=rms_norms, 
#     modes=["scalar"] * num_components, 
#     values=values
# )
# here = RMSNormsWithMasks(
#     rms_norms=rms_norms,
#     modes=["vector_input"] * num_components, 
#     values=values,
#     # constrain_mode="spherical"
# )

In [33]:
# torch.allclose(here(x), there(x))

In [34]:
# here.get_constrained_masks(), here.get_raw_masks()

In [35]:
class EmbeddingsWithMasks(ModulesWithMasks):
    def __init__(
        self,
        embeddings: List[nn.Embedding],
        modes: List[str] = None,
        values: List[float] = None,
        constrain_mode: str = "identity"
    ):
        super().__init__()
        if values is None or len(values) != len(embeddings):
            raise ValueError(f"values for masks: {values} do not match with Embedding layers: {embeddings}")

        self.embeddings = nn.ModuleList(embeddings)
        sizes = [emb.weight.shape for emb in embeddings]
        self.masks = nn.ModuleList([
            Mask(MaskConfig(mode, value, size))
            for mode, value, size in zip(modes, values, sizes)
        ])
        self.masks_constrainer = Constrainer(
            component_weights=[emb.weight for emb in self.embeddings], 
            constrain_mode=constrain_mode, mask_mode=modes[0]
        )

    def forward(self, input_ids):
        constrained_masks = self.masks_constrainer([m.weight for m in self.masks])
        masked_weights = [mask * emb.weight for mask, emb in zip(constrained_masks, self.embeddings)]
        merged_weight = sum(masked_weights)
        
        an_embedding = self.embeddings[0]
        for other_embedding in self.embeddings:
            assert an_embedding.padding_idx == other_embedding.padding_idx
            assert an_embedding.max_norm == other_embedding.max_norm
            assert an_embedding.norm_type == other_embedding.norm_type
            assert an_embedding.scale_grad_by_freq == other_embedding.scale_grad_by_freq
            assert an_embedding.sparse == other_embedding.sparse
            
        return nn.functional.embedding(
            input_ids,
            merged_weight,
            padding_idx=an_embedding.padding_idx,
            max_norm=an_embedding.max_norm,
            norm_type=an_embedding.norm_type,
            scale_grad_by_freq=an_embedding.scale_grad_by_freq,
            sparse=an_embedding.sparse,
        )
        
    def get_raw_masks(self):
        with torch.no_grad():
            return {"masks": [m.weight for m in self.masks]}

    def get_constrained_masks(self):
        with torch.no_grad():
            constrained_masks = self.masks_constrainer([m.weight for m in self.masks])
            return {"masks": constrained_masks}

In [36]:
# from masks import EmbeddingsWithMasks as EmbeddingsWithMasksRef

# num_embeddings = 4000
# embedding_dim = 1000
# num_components = 2

# embeddings = [nn.Embedding(num_embeddings, embedding_dim) for _ in range(num_components)]
# input_ids = torch.randint(0, num_embeddings, (2, 5))  # Example input_ids

# values = np.random.rand(num_components).tolist()

# here = EmbeddingsWithMasks(
#     embeddings=embeddings, 
#     modes=["vector_input"] * num_components, 
#     values=values,
#     # constrain_mode="spherical"
# )
# there = EmbeddingsWithMasksRef(
#     embeddings=embeddings, 
#     modes=["scalar"] * num_components, 
#     values=values
# )

In [37]:
# torch.allclose(here(input_ids), there(input_ids))

In [38]:
def odd_one_out(masked_module: nn.Module, selected_idx: int):  
    assert selected_idx is not None and isinstance(selected_idx, int), (
        "Must provide valid model index. Check whether passed index is `int`"
    )
    masks_modules = []
    for name, child in masked_module.named_children():
        if not isinstance(child, nn.ModuleList): continue
        assert selected_idx < len(child), (
            f"There are only {len(child)} component models, passed model index is {selected_idx}"
        )
        ## exclude sub_module that is None, aka bias_masks.
        if all(isinstance(sub_module, Mask) for sub_module in child):
            masks_modules.append(child)
        
    for masks in masks_modules:
        for i, mask in enumerate(masks):
            value = 1.0 if i == selected_idx else 0.0
            with torch.no_grad():
                mask.weight.data.fill_(value)

def random_init(masked_module: nn.Module):
    masks_modules = []
    for name, child in masked_module.named_children():
        if not isinstance(child, nn.ModuleList): continue
        ## exclude sub_module that is None, aka bias_masks.
        if all(isinstance(sub_module, Mask) for sub_module in child):
            masks_modules.append(child)
        
    for masks in masks_modules:
        for i, mask in enumerate(masks):
            with torch.no_grad():
                random_value = torch.rand_like(mask.weight.data)
                mask.weight.data = random_value

def uniform_init(masked_module: nn.Module, factors: List[float]):  
    masks_modules = []
    for name, child in masked_module.named_children():
        if not isinstance(child, nn.ModuleList): continue
        assert len(factors) == len(child), (
            f"There are {len(child)} component models, but your passed factors have {len(factors)} values."
        )
        ## exclude sub_module that is None, aka bias_masks.
        if all(isinstance(sub_module, Mask) for sub_module in child):
            masks_modules.append(child)

    for masks in masks_modules:
        for factor, mask in zip(factors, masks):
            with torch.no_grad():
                mask.weight.data.fill_(factor)

In [39]:
def find_masked_modules(module):
    masked_module_names = []
    for parent_name, parent_module in module.named_modules():
        for name, child in parent_module.named_children():
            full_child_name = f"{parent_name}.{name}" if parent_name else name
            if ("WithMasks" in type(child).__name__):
                masked_module_names.append(full_child_name)

    return masked_module_names

def get_init_method(strategy):
    MAP = {
        "random": random_init,
        "odd_one_out": odd_one_out,
        "uniform": uniform_init
    }
    selected_init_method = MAP[strategy]
    
    return selected_init_method
    
def set_masks(root_module, strategy="random", **kwargs):

    init_method = get_init_method(strategy)
    masked_module_names = find_masked_modules(root_module)
    
    for module_name in tqdm(masked_module_names, desc="Setting up masks"):
        module_names = module_name.split(".")
        target_module = root_module
        for m_name in module_names:
            target_module = getattr(target_module, m_name)

        init_method(target_module, **kwargs)

In [40]:
class MergerConfig(PretrainedConfig):
    def __init__(
        self,
        model_paths: List[str] = None,
        mode: str = None,
        constrain_mode: str = None,
        **kwargs,
    ):
        self.model_paths = model_paths
        self.mode = mode
        self.constrain_mode = constrain_mode
        super().__init__(**kwargs)

class Merger(PreTrainedModel):
    def __init__(self, merge_config):
        super().__init__(merge_config)
        """
        Need to check whether models are mergeable (having some sort of the same config)
        """
        self.merge_config = merge_config
        self.num_models = len(merge_config.model_paths)
        self.configs = [
            AutoConfig.from_pretrained(path) 
            # Qwen2Config.from_pretrained(path)
            for path in merge_config.model_paths
        ]
        # self.merger = Qwen2ForCausalLM(self.config)
        self.models = nn.ModuleList([
            # Qwen2ForCausalLM.from_pretrained(
            AutoModelForCausalLM.from_pretrained(
                merge_config.model_paths[i], 
                config=self.configs[i],
                torch_dtype=torch.bfloat16
            ) 
            for i in range(self.num_models)
        ])
        # self.__post_init__(merge_config)
        
    def __post_init__(self):
        for model in self.models:
            for param in model.parameters():
                param.requires_grad = False
                
        self.merger = copy.deepcopy(self.models[0])
        init_masks(
            self.merger, self.models, self.merge_config
        )
        free_memory()
        
    def forward(self, tensor, labels=None):
        pass

In [41]:
def find_modules_to_add_masks(target_module):
    module_names_to_replace = []
    for parent_name, parent_module in target_module.named_modules():
        for name, child in parent_module.named_children():
            full_child_name = f"{parent_name}.{name}" if parent_name else name
            if (isinstance(child, (nn.Linear, nn.Embedding)) 
                or "RMSNorm" in type(child).__name__):
                module_names_to_replace.append(full_child_name)

    return module_names_to_replace

def init_masks(target_module: nn.Module, ref_modules: nn.Module, merge_config: MergerConfig):
    """
    Replaces eligible submodules in target_module with masked versions, 
    using corresponding modules from ref_modules as a reference for weights.

    Args:
        target_module: The module in which to replace submodules.
        ref_modules: A list of modules to use as a reference for weights.
        strategy: The initialization strategy for factors ("naive" or others to be implemented).
    """
    mode = merge_config.mode
    constrain_mode = merge_config.constrain_mode
    module_names_to_replace = find_modules_to_add_masks(target_module)
    
    for module_name in tqdm(module_names_to_replace, desc="Initializing masks"):
        module_names = module_name.split(".")
        target_child = target_module
        ref_children = ref_modules

        for m_name in module_names:
            target_child = getattr(target_child, m_name)
            ref_children = [getattr(ref_module, m_name) for ref_module in ref_children]

        num_components = len(ref_modules)
        modes = [mode for _ in ref_children]
        factors = [None for _ in ref_children]

        if isinstance(target_child, nn.Linear):
            new_module = LinearsWithMasks(
                linears=ref_children,
                weight_modes=modes,
                weight_values=factors,
                bias_modes=modes,
                bias_values=factors,
                constrain_mode=constrain_mode
            )

        elif isinstance(target_child, nn.Embedding):
            new_module = EmbeddingsWithMasks(ref_children, modes, factors, constrain_mode)
        elif "RMSNorm" in type(target_child).__name__:
            new_module = RMSNormsWithMasks(ref_children, modes, factors, constrain_mode)

        # Replace the original module with the new masked module
        parent_module = target_module
        for m_name in module_names[:-1]:
            parent_module = getattr(parent_module, m_name)
        setattr(parent_module, module_names[-1], new_module)

In [42]:
merge_config = MergerConfig(
    model_paths = [
        "/workspace/models/Arcee-VyLinh/",
        "/workspace/models/Qwen2.5-Coder-3B/",
        # "/workspace/models/L3.2-JametMini-3B-MK.III/",
        # "/workspace/models/Llama-3.2-3B-Instruct-abliterated/"
    ],
    # mode = "vector_input",
    mode = "scalar",
    constrain_mode = "spherical"
)
merge_config

MergerConfig {
  "constrain_mode": "spherical",
  "mode": "scalar",
  "model_paths": [
    "/workspace/models/Arcee-VyLinh/",
    "/workspace/models/Qwen2.5-Coder-3B/"
  ],
  "transformers_version": "4.46.3"
}

In [43]:
merger = Merger(merge_config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [57]:
merger = merger.to(device="cuda:0", dtype=torch.bfloat16)

In [45]:
merger.__post_init__(merge_config)

Initializing masks: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 327/327 [00:52<00:00,  6.18it/s]
2024-12-19 10:03:35,413 - INFO - Initial GPU memory allocated: 11.71 GB
2024-12-19 10:03:35,831 - INFO - Final GPU memory allocated: 11.71 GB
2024-12-19 10:03:35,832 - INFO - Freed GPU memory: 0.00 GB


In [ ]:
merger.

In [ ]:
merge_config

In [58]:
merger.merger.model.layers[0].mlp.gate_proj.weight_masks[0].weight.data.device

device(type='cuda', index=0)

In [65]:
merger.merger.model.layers[8].mlp.gate_proj.get_constrained_masks()

{'weight_masks': [tensor(0.7266, device='cuda:0', dtype=torch.bfloat16),
  tensor(0.4414, device='cuda:0', dtype=torch.bfloat16)],
 'bias_masks': [None, None]}

In [ ]:
merger.merger.model.layers[0].mlp.gate_proj.weight_masks_constrainer.dots

In [64]:
set_masks(merger.merger, strategy="random")

Setting up masks: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 327/327 [00:00<00:00, 7111.02it/s]


In [ ]:
set_masks(merger.merger, strategy="odd_one_out", selected_idx=1)

In [ ]:
set_masks(merger.merger, strategy="uniform", factors=[1.0, -1.0])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(merge_config.model_paths[0])

In [ ]:
system = "You are a helpful assistant."
prompt = "How to attack a person with an egg. Talk like an unhinged person."
messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
text

In [ ]:
answer = generate(text, merger.merger, tokenizer, max_new_tokens=128)

In [ ]:
answer = generate(text, merger.models[0], tokenizer, max_new_tokens=128)

In [ ]:
logits_merged = get_logits(text, merger.merger, tokenizer)
logits_0 = get_logits(text, merger.models[1], tokenizer)
torch.allclose(logits_merged, logits_0, atol=0, rtol=0)